<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates AWS Access Key Rotation for a user using unSkript legos.</b>
</div>

<br>

<center><h2>AWS Access Key Rotation</h2></center>

# Steps Overview
    1) Create a new Access Key for the user
    2) Update the status of the old Access Key to "Inactive"
    3) Check if there are 2 Access Keys one with status "Active" and another with "Inactive"
    4) Delete the Inactive Access Key

In this lego we will create a new Access Key for the user as given in the input parameters

In [6]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
import pandas as pd
from typing import List,Dict
import pprint


from beartype import beartype
@beartype
def aws_create_access_key_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_create_access_key(
    handle,
    aws_username: str
) -> Dict:
    retVal = {}
    temp_list = []
    iamClient = handle.client('iam')
    result = iamClient.create_access_key(UserName=aws_username)
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_username": "aws_username"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_create_access_key, lego_printer=aws_create_access_key_printer, hdl=hdl, args=args)

In this lego we will update the status of the old Access Key to "Inactive". This step is required to delete the old access key as one user cannot have 2 Access Keys. Hence, we need the old AWS Access Key ID, status to set and AWS username as inputs.

In [7]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field, SecretStr
import pandas as pd
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_update_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key status successfully changed")
    pprint.pprint(output)

@beartype
def aws_update_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
    status: str
) -> Dict:
    retVal = {}
    temp_list = []
    iamClient = handle.client('iam')
    result = iamClient.update_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id, Status=status)
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_access_key_id": "aws_access_key_id",
    "aws_username": "aws_username",
    "status": "status"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_update_access_key, lego_printer=aws_update_access_key_printer, hdl=hdl, args=args)

Here we will list all the Access Keys allotted to the given user. The new one should have the status as "Active" and the old one should have the status as "Inactive".

In [10]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict,List
import pprint


from beartype import beartype
@beartype
def aws_list_access_keys(
    handle,
    aws_username: str
) -> Dict:
    retVal = {}
    temp_list=[]
    iamClient = handle.client('iam')
    result = iamClient.list_access_keys(UserName=aws_username)
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


def aws_list_access_keys_printer(output):
    if output is None:
        return
    pprint.pprint(output)

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_username": "aws_username"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_access_keys, lego_printer=aws_list_access_keys_printer, hdl=hdl, args=args)

In this lego we will delete the the old (Inactive) Access Key.

In [9]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
import pandas as pd
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_delete_access_key_printer(output):
    if output is None:
        return
    pprint.pprint("Access Key successfully deleted")
    pprint.pprint(output)


@beartype
def aws_delete_access_key(
    handle,
    aws_username: str,
    aws_access_key_id: str,
) -> Dict:
    iamClient = handle.client('iam')
    result = iamClient.delete_access_key(UserName=aws_username, AccessKeyId=aws_access_key_id)
    retVal = {}
    temp_list = []
    for key, value in result.items():
        if key not in temp_list:
            temp_list.append(key)
            retVal[key] = value
    return retVal


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "aws_access_key_id": "aws_access_key_id",
    "aws_username": "aws_username"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_access_key, lego_printer=aws_delete_access_key_printer, hdl=hdl, args=args)

<p>In this Runbook, we demonstrated the use of unSkript's AWS legos to perform Access key rotation. To view the full platform capabilities of unSkript please visit <a href="https://unskript.com">https://unskript.com</a></p>